# COCO to YOLO Conversion Tool

## Dependencies

### Install packages

In [24]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import packages

In [25]:
import json
import os
import shutil

from tqdm import tqdm

## Conversion Tool

### Conversion Function

In [26]:
def convertCOCOtoYOLO(jsonPath, outputPath, categories):
    print(f"Reading JSON from: {jsonPath}")
    print(f"Output directory: {outputPath}")

    # If the output directory exists, delete it
    if os.path.exists(outputPath):
        shutil.rmtree(outputPath)

    # Create the new output directory
    os.makedirs(outputPath)

    with open(jsonPath) as f:
        data = json.load(f)

    images = data["images"]

    # Set up the progress bar
    with tqdm(total=len(images), desc="Converting COCO to YOLO", unit="image") as pbar:
        for image in images:
            imageID = image["id"]
            fileName = image["file_name"]

            # Intercept the path and remove any leading directory
            fileName = os.path.basename(fileName)  # Keeps only the file name, removing directories

            width = image["width"]
            height = image["height"]

            annotations = [
                ann
                for ann in data["annotations"]
                if ann["image_id"] == imageID and 0 <= ann["category_id"] <= categories
            ]

            outputFilePath = os.path.join(outputPath, f"{os.path.splitext(fileName)[0]}.txt")

            # Ensure the directory for the output file exists
            outputFileDir = os.path.dirname(outputFilePath)
            if not os.path.exists(outputFileDir):
                os.makedirs(outputFileDir)

            if annotations:
                with open(outputFilePath, "w") as f:
                    for ann in annotations:
                        categoryID = ann["category_id"]
                        bbox = ann["bbox"]
                        xCenter = (bbox[0] + bbox[2] / 2) / width
                        yCenter = (bbox[1] + bbox[3] / 2) / height
                        w = bbox[2] / width
                        h = bbox[3] / height
                        f.write(f"{categoryID} {xCenter} {yCenter} {w} {h}\n")

            # Update the progress bar
            pbar.update(1)

### Conversion Operation

In [27]:
# Call the function with the JSON files and respective category counts
#convertCOCOtoYOLO("./annotations/coco_train.json", "./outputs/coco_train", 13)
#convertCOCOtoYOLO("./annotations/coco_validation.json", "./outputs/coco_validation", 13)
#convertCOCOtoYOLO("./annotations/coco_test.json", "./outputs/coco_test", 13)
#convertCOCOtoYOLO("./annotations/thermalCoco_train.json", "./outputs/thermalCoco_train", 15)
#convertCOCOtoYOLO("./annotations/thermalCoco_validation.json", "./outputs/thermalCoco_validation", 15)
#convertCOCOtoYOLO("./annotations/thermalCoco_test.json", "./outputs/thermalCoco_test", 15)